- Please run `sh install.sh` in the terminal first.

In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import os
import gc
import time
import numpy as np
import pandas as pd
import tensorflow as tf
# tf.debugging.set_log_device_placement(True)

%matplotlib inline
from keras.preprocessing.image import array_to_img
import copy
import seaborn as sns
import cv2 as cv
import pickle
import tensorflow_probability as tfp
from tensorflow.keras.layers import AveragePooling2D
import shutil
import sys 

sys.path.append("../models/GPA")
sys.path.append("../")
from gpa import GPA
from utils import *
from simu_auxiliary import *

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 Physical GPUs, 1 Logical GPUs


2024-12-09 06:33:01.571218: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-09 06:33:04.184640: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 17003 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:9b:00.0, compute capability: 8.0


In [3]:
def MSE(x, y):
    """Compute the Mean Square Error."""
    mse = tf.reduce_mean((x - y)**2)
    return mse

## Hyper-parameters

In [4]:
T = 100                    # the number of replications
N_list = [100, 500, 1000]  # the number of images
p = 540                    # image height
q = 960                    # image width
M = p * q                  # the number of pixel locations
G = 500                    # the number of grid points
test_size = 100            # the number of test images
truncate_width = 3         # filter size for kernel smoothing

## Distribution

In [5]:
# std & mean
sigma = 0.2  # standard deviation
mean = np.load('./mean-540.npy').reshape([1, p, q, 1])  # mean
avg_pool_2d_mean = AveragePooling2D(pool_size=(5, 5), strides=1, padding="same")
mean = tf.reshape(avg_pool_2d_mean(mean), (1, p, q))

# grid points
tick_list = np.random.uniform(size=G)
tick_tensor = tf.concat([tf.ones([1, p, q]) * tick for tick in tick_list], axis=0)

# oracle dsitrbution
f_true = compute_true_density(tick_list, mean, sigma)
tfd = tfp.distributions
dist = tfd.TruncatedNormal(loc=mean, scale=sigma, low=[0.], high=[1.])

2024-12-09 06:33:06.494264: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101


True density shape: (500, 540, 960)


## Run Experiments

In [6]:
experiment_stats = {"N": [],
                    "t": [],
                    "CD_mse": [],
                    "DS_mse": [],
                    "GPA_CD_mse": [],
                    "GPA_DS_mse": [],
                    "DS_mse": [],
                    "CD_time": [],
                    "DS_time": [],
                    "GPA_CD_time": [],
                    "GPA_DS_time": []}

In [ ]:
for N in N_list:
    print(f"============= N:{N} =============")
    alpha = np.log(p * q) / np.log(N)  # alpha
    path = f'../../simulate_img_N={N}_GPU1'  # path to save N simulate image

    # optimal bandwidth
    bandwidth, bandwidth_star = compute_optimal_bandwidths(N, M, sigma)
    location_weight_tensor = compute_location_weight(p, q, bandwidth, truncate_width)
    location_weight_tensor = tf.reshape(location_weight_tensor, [truncate_width, truncate_width, 1, 1])
    location_weight_tensor = tf.cast(location_weight_tensor, tf.float32)

    for t in range(T): # repliacte
        print(f"============= The {t}th replication with N samlpes:{N} =============")
        seed = N + t
        tf.random.set_seed(seed) # set random seed in one replication of N
        np.random.seed(seed)
        CD_mse = 0
        DS_mse = 0
        GPA_CD_mse = 0
        GPA_DS_mse = 0
        CD_time = 0
        DS_time = 0
        GPA_CD_time = 0
        GPA_DS_time = 0
        
        # generate N simulation images
        generate_simulate_data(path, N, mean, sigma)
        
        # pre-computed CD and DS matrix
        CD_matrix = compute_CD_matrix(path, N, G, p, q, bandwidth, tick_tensor)
        DS_matrix = compute_DS_matrix(CD_matrix, location_weight_tensor)
        CD_matrix = tf.squeeze(CD_matrix)
        DS_matrix = tf.squeeze(DS_matrix)
        
        # density comparison
        for test_i in range(test_size): # 生成测试图片
            print(f"[TEST] image No.{test_i}")
            test_img = tf.ones([p, q]) * np.random.uniform()
            oracle_test = tf.squeeze(dist.prob(test_img))
            
            # CD estimation
            print(f"[TEST] compute CD estimator")
            t1 = time.time()
            CD_test = test_CD(p, q, test_img, bandwidth, path)
            CD_test = tf.reshape(CD_test, (1, p, q))
            t2 = time.time()
            CD_time += (t2 - t1) / test_size
            
            # DS estimation
            print(f"[TEST] compute DS estimator")
            t3 = time.time()
            DS_test = compute_DS_matrix(CD_test, location_weight_tensor)
            t4 = time.time()
            DS_time += (t4 - t3 + t2 - t1) / test_size
            
            # GPA-CD estimation
            print(f"[TEST] compute GPA-CD estimator")
            t1 = time.time()
            Omega2_star = K_tf(tick_tensor - test_img, bandwidth_star)
            Omega1_star = Omega2_star * CD_matrix
            Omega1_star = tf.reduce_sum(Omega1_star, axis=0)
            Omega2_star = tf.reduce_sum(Omega2_star, axis=0)
            GPA_CD_test = Omega1_star / Omega2_star
            t2 = time.time()
            GPA_CD_time += (t2 - t1) / test_size
            
            # GPA-DS estimation
            print(f"[TEST] compute GPA-DS estimator")
            t1 = time.time()
            Omega2_star = K_tf(tick_tensor - test_img, bandwidth_star)
            Omega1_star = Omega2_star * DS_matrix
            Omega1_star = tf.reduce_sum(Omega1_star, axis=0)
            Omega2_star = tf.reduce_sum(Omega2_star, axis=0)
            GPA_DS_test = Omega1_star / Omega2_star
            t2 = time.time()
            GPA_DS_time += (t2 - t1) / test_size

            ###################### Compute MSE ####################
            # 1. MSE of the CD estimator
            CD_test = tf.squeeze(CD_test)
            CD_mse += MSE(oracle_test, CD_test) / test_size
            # 2. MSE of the DS estimator
            DS_test = tf.squeeze(DS_test)
            DS_mse += MSE(oracle_test, DS_test) / test_size
            # 3. MSE of the GPA-CD estimator
            GPA_CD_test = tf.squeeze(GPA_CD_test)
            GPA_CD_mse += MSE(oracle_test, GPA_CD_test) / test_size
            # 4. MSE of the GPA-DS estimator
            GPA_DS_test = tf.squeeze(GPA_DS_test)
            GPA_DS_mse += MSE(oracle_test, GPA_DS_test) / test_size
            print(f"Sample size N:{N}")
            print(f"CD MSE: {CD_mse.numpy():.8f}")
            print(f"DS MSE: {DS_mse.numpy():.8f}")
            print(f"GPA_CD MSE: {GPA_CD_mse.numpy():.8f}")
            print(f"GPA_DS MSE: {GPA_DS_mse.numpy():.8f}")
        
        experiment_stats["N"].append(N)
        experiment_stats["t"].append(t)
        experiment_stats["CD_mse"].append(CD_mse.numpy())
        experiment_stats["DS_mse"].append(DS_mse.numpy())
        experiment_stats["GPA_CD_mse"].append(GPA_CD_mse.numpy())
        experiment_stats["GPA_DS_mse"].append(GPA_DS_mse.numpy())
        experiment_stats["CD_time"].append(CD_time)
        experiment_stats["DS_time"].append(DS_time)
        experiment_stats["GPA_CD_time"].append(GPA_CD_time)
        experiment_stats["GPA_DS_time"].append(GPA_DS_time)
        experiment_stats_csv = pd.DataFrame(experiment_stats)
        experiment_stats_csv.to_csv(f'./simulation(N={N}).csv')

## Experiment Results

In [8]:
data1 = pd.read_csv("./simulation(N=100).csv")
data2 = pd.read_csv("./simulation(N=500).csv")
data3 = pd.read_csv("./simulation(N=1000).csv")

In [9]:
data1.head()

,Unnamed: 0,N,t,CD_mse,DS_mse,GPA_CD_mse,GPA_DS_mse,CD_time,DS_time,GPA_CD_time,GPA_DS_time
0,0,100,0,0.121442,0.017991,0.120664,0.017967,0.406773,0.408354,0.000917,0.000474
1,1,100,1,0.117676,0.016189,0.116900,0.016013,0.405915,0.407258,0.000520,0.000475
2,2,100,2,0.128435,0.017265,0.127516,0.017177,0.409783,0.411178,0.000521,0.000476
3,3,100,3,0.119510,0.016508,0.118746,0.016487,0.398344,0.399704,0.000517,0.000474
4,4,100,4,0.118949,0.017294,0.118441,0.017419,0.410040,0.411441,0.000528,0.000486


In [10]:
data1.GPA_DS_time.mean(), data2.GPA_DS_time.mean(), data3.GPA_DS_time.mean()

(0.000495985651016188, 0.00037917010784144505, 0.000377491402625987)

In [11]:
data1.DS_time.mean(), data2.DS_time.mean(), data3.DS_time.mean()

(0.4096721539735795, 0.9244988394737245, 1.8066494903802874)

In [12]:
data1.GPA_CD_time.mean(), data2.GPA_CD_time.mean(), data3.GPA_CD_time.mean()

(0.000545032715797376, 0.00041922466754908404, 0.00042484087944026096)

In [13]:
data1.CD_time.mean(), data2.CD_time.mean(), data3.CD_time.mean()

(0.40827391393184653, 0.9233673335790634, 1.80542599709034)